## ДЗ 7

In [2]:
import torch.nn as nn
import torch

In [3]:
vocab_size = 100
embedding_dim = 16
hidden_dim = 32

class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(lstm_out)
    
lstm_init = LSTMFixedLen(vocab_size, embedding_dim, hidden_dim)
optimizer = torch.optim.Adam(lstm_init.parameters(), lr=0.00051)
#criterion = nn.CrossEntropyLoss() 
criterion = nn.BCELoss() 

## ДЗ 8

In [30]:
from torch.utils.data import DataLoader, Dataset
class Data(Dataset):
    def __init__(self, func, min, max, num):
        self.num = num
        self.x = np.random.rand(num) * (max - min) + min
        self.func = func

    def __getitem__(self, idx):
        point = self.x[idx]
        # return torch.FloatTensor([point, self.func(point)])
        return torch.Tensor([point, self.func(point)])

    def __len__(self):
        return self.num

In [31]:
class Generator(nn.Module):
    def __init__(self, latent_dim, hidden_dim):
        super().__init__()

        self.linear_1 = nn.Linear(latent_dim, hidden_dim)
        self.linear_2 = nn.Linear(hidden_dim, 2)

    def forward(self, x):
        x = self.linear_1(x)
        x = torch.tanh(x)
        x = self.linear_2(x)      
        return x

# В предыдущей серии


<img src="https://drive.google.com/uc?export=view&id=12ZNQ57Fj9arjbwRR-jS6atV8o_IXTMwI"/>


Мы посмотрели на задачу классификации текстов. Но есть ряд более сильных подходов, которые лучше показывать через задачу генерации


# Генерация текстов, encoder-decoder

<img src="https://drive.google.com/uc?export=view&id=1vq571MgxcLaKXyE5E0-qxpauwIZmSc1T"/>


Данная архитектура называется seq2seq, простыми словами выглядит она следующим образом:
<img src="https://drive.google.com/uc?export=view&id=1MAi5s0Jl0QdvOFU_ILJqQLk5stsPLHYF"/>


## Attention

Примеры из статьи [Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation](https://arxiv.org/abs/1609.08144).

<img src='https://drive.google.com/uc?export=view&id=1uOHQX8AnPGKY6HEDOTRJzVxCiKAHLoJy' width=450>

Нужно сжать всю информацию о предыдущих словах в один блок, который представляет собой внутренний слой - переходное состояние из кодировщика в декодировщика, что очень сложная задача, которая приводит к недообучению.

И одно из возможных решений - это настакать слои LSTM, один слой LSTM создает вход для другого слоя LSTM:

<img src='https://drive.google.com/uc?export=view&id=1yrBxb5DXZA-LNwNWQdmuF4TNNXqTfNpJ' width=450>

Но эту вещь очень тяжело обучать, если добавлять всё больше LSTM слоем, то снова встречаемся с проблемой затухающих градиентов, плюсом, всё равно нужно сжимать всю информацию в последние блоки - это наш bottleneck (узкое место, горлышко бутылки) между кодировщик и декодировщиком.


И решение этой проблемы - Attention слой. Вместо того, чтобы сжимать всю информацию из всех временных ячеек в одно скрытое состояние, можно дать доступ декодировщику ко всей истории. Но получаем очень слишком много информации, которую нужно учитывать, поэтому мы будем обращать свое внимание только на подвыборку этих ячеек.

Будем учитывать, какая часть предложения на английском языке важна для предсказания слова на французском языке. Подсчет такого распределения достигается получением оценки релевантности каждого слова в предложении для получения нового слова.

<img src='https://drive.google.com/uc?export=view&id=1GAVWyShMSXELaz1f-_e4c9rAeDoJ3QE_' width=450>


**Как использовать эти оценки релевантности?**

Подсчитываются оценки для каждого скрытого состояния и образуют собой взвешанную сумму внимания. А потом эта сумма передается, как еще один вход в декодировщик.

<img src='https://drive.google.com/uc?export=view&id=1a72Ake6Ai2SdkZ9cQpqmq6-ebjblcQ34' width=450>

Можно [провизуализировать](https://distill.pub/2016/augmented-rnns/#attentional-interfaces), какое внимание уделяется каждому слову из предложения:

<img src='https://drive.google.com/uc?export=view&id=1OJ4vgTHPxGMzcR8Ob8An--FFUlnZabQ3' width=500>

А ещё слои внимания можно добавлять и не только к текстам, но и к картинкам.

<img src='https://drive.google.com/uc?export=view&id=1i48NmZMXUneeUwW339tSgvrGU_VvLh9H'>



Теперь более детально, как рассчитать внимание.


<table>
<tr>
<td>
<img src="https://drive.google.com/uc?export=view&id=12oHuf30zRZR_rEQ3P7Y599qZVexH50C1"/>
</td>
<td>
<img src="https://drive.google.com/uc?export=view&id=1XOmhFw2RvweVYl9m7pUN0QQD5R0Kj_yl"/>
</td>
</td>
</table>


- h(t): скрытое состояние декодера
- c(t): вектор контекста, который подается на вход
- y(t): текущий таргет
- $\bar{h}(t)$: скрытое состояние attention
- a(t): скор нормализации


Зачем нужен скор нормализации? - пытаемся сравнить похожесть текущего скрытого состояния и скрытого состояния из прошлого и понять, на что обращать внимание



#### Attention Decoder

Туториал: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

Но если передается только один контекстный вектор с энкодера в декодер, то в нём должна быть вся информация, что естественно очень тяжело выучить.

Attention позволяет декодеру сфокусироваться на разных частях выхода энкодера. Сначала считаются веса attention'a. Они перемножаются на выходы энкодера, чтобы создать взвешенную комбинацию. 

Вычисление весов внимание можно сделать через полносвязную сеть, используя входы декодера и скрытые состояния.

Из-за того, что предложения разной длины, невозможно обучать такой слой, так что нужно выбрать максимальную длину предложения входной последовательности и так обучаться. Предложения максимальной длины будут использовать все веса внимания, а короткие предложения только часть.

<img src='https://drive.google.com/uc?export=view&id=1Wde98PnXvXEw9jHBca524pEy45yBfQzd'>





In [4]:
from torch import nn

MAX_LENGTH = 20

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, vocab_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.vocab_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.vocab_size)

    def forward(self, input, hidden, encoder_outputs):
        # input - один вход на декодер (сымый первый - это токен SOS)
        embedded = self.embedding(input).view(1, -1)  # (1, hidden_size)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded, hidden), 1)),
            dim=1)  # (1, max_length)

        # encoder_outputs (max_length, hidden_size)
        attn_applied = torch.matmul(attn_weights, encoder_outputs)  # (1, hidden_size)

        output = torch.cat((embedded, attn_applied), 1) # (1, hidden_size*2)
        output = self.attn_combine(output)  # (1, hidden_size)

        output = F.relu(output)
        output, hidden = self.gru(output.unsqueeze(0), hidden)  # (1, 1, hidden_size)

        output = F.log_softmax(self.out(output[0]), dim=1)  # (1, vocab_size)
        return output, hidden, attn_weights

# Transformers

### Общее представление

[The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)


В трансформере находится энкодер и декодер, которые в свою очередь представляют стэк энкодеров и декодеров (в оригинальной статье используется по 6 каждого).   

<img src='https://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png' width=500>

Энкодеры все идентичны по структуре, хотя у них разные веса. Каждый можно поделить на два подслоя:

<img src='https://jalammar.github.io/images/t/Transformer_decoder.png' width=500>

Вход энкодера сначала поступает на слой self-attention'a - слой, который позволяет энкодеру смотреть на другие слова во входной последовательности, чтобы более качественно закодировать слово.  
Выходы из self-attention'a скармливаются в сетку с прямым проходом.

У декодера есть такие же два слоя (self-attention и feed forward сетка), но между ними добавляется слой внимания, который фокусируется на релевантных частях входной последовательности.

### Устройство энкодера

Теперь добавим ещё больше деталей. В самом начале нужно перевести входные слова в эмбеддинги. Каждое слово будет представлять из себя вектор размерности 512.  
Эмбеддинги передаются только в самом первом энкодере, дальше передаются уже выходы из промежуточных энкодеров. Размер таких последовательностей - это гиперпараметр, который можем установить размером самого длинного предложения в обучающей выборке.


<img src='https://jalammar.github.io/images/t/encoder_with_tensors.png' width=500>

Начинает проглядываться ключевая особенность трансформера - слово в последовательности протекает по своему собственному пути в энкодере. Есть зависимости между этими дорожками в слое self-attention'a, а вот в feed-forward слое таких зависимостей нет, поэтому такие вычисления могут совершаться параллельно.


Первый шаг в вычислении self-attention - создание трех векторов из каждого входного эмбеддинга. Для каждого слова создается вектор Query, вектор Key и вектор Value. Эти векторы создаются путем умножения эмбеддинга на три матрицы, которые обучаются.
- Query - слово, с которого смотрим на всё остальное
- Key - слово, на которое смотрим
- Value - здесь содержится смысл слова

При перемножении Query на Key мы понимаем насколько слово с Key релевантно слову с Query.

<img src='http://jalammar.github.io/images/t/transformer_self_attention_vectors.png' width=500>

Все шаги вместе:

<img src='https://jalammar.github.io/images/t/self-attention-output.png' width=500>

Деление на $\sqrt{d}$ (квадратный корень из размерности Key векторов. для того, чтобы были более стабильные градиенты.

<img src='http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png' width=400>

### Multi-head self-attention


<img src='http://jalammar.github.io/images/t/transformer_attention_heads_qkv.png' width=600>

В трансформерах используется не один слой self-attention'a, а несколько, так называемый механизм multi-headed attention. Это улучшает обычный слой внимания двумя путями:
1. Он расширяет возможность модели фокусироваться на разных позициях. Иногда внимание может упасть на само это слово, а на другие не будет обращать внимание. (*Животное не переходило дорогу, потому что оно устало*)

2. Он позволяет слою внимания иметь несколько признаковых описаний. Раз голов внимания больше, то и будет больше наборов с матрицами весов, которые как-то переводят наши слова в векторы для энкодера/декодера. Получаем, что после обучения, каждый такой набор матриц позволит переводить входной эмбеддинг в разные пространства.

<img src='https://jalammar.github.io/images/t/transformer_attention_heads_z.png' width=500>

В примере 8 голов слоя внимания, но feed forward сеть не ожидает столько входов, поэтому нужно сконкатенировать выходы, перемножить на матрицу весов и получить вектор, который является собирательным описанием всех голов внимания. <img src='https://jalammar.github.io/images/t/transformer_attention_heads_weight_matrix_o.png' width=600>

Еще одна деталь в трансформерах:  
они добавляют позиционные векторы ко входным эмбеддингам, эти векторы следуют паттерны, которые изучает модель, чтобы помогать определять позиции каждого слова или дистанции между словами в последовательностях.  
<img src='https://jalammar.github.io/images/t/transformer_positional_encoding_vectors.png' width=600>

### Декодер

Выход последнего энкодера трансформируется в набор векторов внимания, они используются в декодере в слое внимания, который помогает декодеру фокусироваться на нужных местах во входных последовательностях.

Такие шаги повторяются, пока не будет получен специальный символ, показывающий, что декодер закончил свой вывод. Выход каждого шага идет в следующий декодер и так же здесь, как и в части с энкодерами, добавляет позиционный векторы в начало декодирования для обозначения позиции каждого слова.

<img src='https://jalammar.github.io/images/t/transformer_decoding_2.gif' width=700>

# BERT


Идея в основе BERT лежит очень простая: давайте на вход нейросети будем подавать фразы, в которых 15% слов заменим на [MASK], и обучим нейронную сеть предсказывать эти закрытые маской слова.


А для того, чтобы нейросеть научилась понимать соотношения между разными предложениями, дополнительно обучим ее предсказывать, является ли вторая фраза логичным продолжением первой. Или это какая-то случайная фраза, не имеющая никакого отношения к первой.


Так, для двух предложений: "Я пошел в магазин." и "И купил там молоко.", нейросеть должна ответить, что это логично. А если вторая фраза будет "Карась небо Плутон", то должна ответить, что это предложение никак не связано с первым.


Обучив таким образом нейронную сеть на корпусе текстов из Wikipedia и сборнике книг BookCorpus в течении 4 дней на 16 TPU, получили BERT.



Как это делается на практике:


<img src="https://drive.google.com/uc?export=view&id=1ENbcSICPN7YRmJKu3NvTBE1z9YLEtHMR"/>


Кодирует 15 процентов слов [MASK] и учим трансформеры + fully connected слои для классификации. Далее, формируем сет, где 50% предложений идут последовательно и 50% рандомно. Учим классификатор (заменяем первый и последний токены предложения, чтобы модель могла их отличить). Учится софтмакс


<img src="https://drive.google.com/uc?export=view&id=1UUd8vjf7CbLoRhgALuUeE7jaCnLrnv5f"/>


Как проходит обучение:

<img src='https://neerc.ifmo.ru/wiki/images/thumb/b/bc/BERT_pre-training.png/350px-BERT_pre-training.png'>


## GPT‑3 (Generative Pre‑trained Transformer 3) 

Обучена на 500 миллиардах слов и включает 175 миллиардов параметров. Модель способна качественно генерировать целые статьи и развёрнуто отвечать на вопросы, в том числе и достаточно узкоспециализированные, например из области генетики, а также анализировать строки, делая выводы из неструктурированных данных.

In [5]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter

import pandas as pd

In [6]:
!pip install transformers

In [7]:
from transformers import pipeline

unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Pytorch is [MASK] than Tensorflow.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.5470118522644043,
  'token': 5514,
  'token_str': 'faster',
  'sequence': 'pytorch is faster than tensorflow.'},
 {'score': 0.13801756501197815,
  'token': 12430,
  'token_str': 'slower',
  'sequence': 'pytorch is slower than tensorflow.'},
 {'score': 0.06793206930160522,
  'token': 16325,
  'token_str': 'simpler',
  'sequence': 'pytorch is simpler than tensorflow.'},
 {'score': 0.04572606459259987,
  'token': 3760,
  'token_str': 'smaller',
  'sequence': 'pytorch is smaller than tensorflow.'},
 {'score': 0.04492785036563873,
  'token': 3469,
  'token_str': 'larger',
  'sequence': 'pytorch is larger than tensorflow.'}]

https://huggingface.co/models

In [8]:
sentiment = pipeline("text-classification", model='Skoltech/russian-inappropriate-messages')
sentiment("Этот ресторан очень плохой")

[{'label': 'LABEL_0', 'score': 0.9979189038276672}]

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [9]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2023-11-20 19:44:25--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Распознаётся drive.google.com (drive.google.com)… 64.233.164.194, 2a00:1450:4010:c0a::c2
Подключение к drive.google.com (drive.google.com)|64.233.164.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


303 See Other
Адрес: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jdsc3ife8be28gmlaed1i3idk09nc9qp/1700498625000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=a89ed30e-5770-4152-9af3-38602d4e480b [переход]
Предупреждение: в HTTP шаблоны не поддерживаются.
--2023-11-20 19:44:30--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jdsc3ife8be28gmlaed1i3idk09nc9qp/1700498625000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=a89ed30e-5770-4152-9af3-38602d4e480b
Распознаётся doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)… 64.233.161.132, 173.194.73.132, 2a00:1450:4010:c05::84
Подключение к doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)|64.233.161.132|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 10733963 (10M) [application/zip]
Сохранение в: «data.zip»

d

In [10]:
!unzip data.zip

Archive:  data.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C


In [11]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [12]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [13]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [14]:
idx = 11
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

мартовские путёвки дорожают на глазах. только пару дней назад были за 66, уже 86 о_О
label is 0


[{'label': 'LABEL_0', 'score': 0.9420949816703796}]

In [15]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

- input_ids - закодированные айди токенов
- attention_mask нужна для того, чтобы PAD символы не проходили через трансформер

In [16]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[  101, 14337, 33930, 77572, 10520, 84964, 15065, 33917,   102,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [17]:
tokenizer.ids_to_tokens[14337], tokenizer.ids_to_tokens[77572], tokenizer.ids_to_tokens[33930], tokenizer.ids_to_tokens[14337]

('При', 'текста', '##мер', 'При')

In [18]:
# tokenizer.get_vocab()

In [19]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] Пример текста для токенизации [SEP] [PAD]


Препроцессинг для берта можно не делать, т.к. он самостоятельно умеет работать со знаками препинания, с разными регистрами и т.д.

In [20]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [21]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [22]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from torch import nn
from transformers import BertModel


class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

'cuda'

In [25]:
model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.linear.parameters(), lr=0.001)  # неполное обучение

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [26]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [27]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

  0%|                                                  | 0/2836 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████████| 2836/2836 [02:24<00:00, 19.62it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- A

Epochs: 1 | Train Loss:  0.011         | Train Accuracy:  0.567         | Val Loss:  0.011         | Val Accuracy:  0.584


  0%|                                                  | 0/2836 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████████| 2836/2836 [02:23<00:00, 19.81it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- A

Epochs: 2 | Train Loss:  0.011         | Train Accuracy:  0.574         | Val Loss:  0.011         | Val Accuracy:  0.583


## Домашнее задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

## Дополнительные материалы:
 Фреймворк для упрощения обучения моделей (аналог keras): https://github.com/Lightning-AI/lightning

Аугментации : https://albumentations.ai/docs/api_reference/pytorch/transforms/

Метрики : https://github.com/Lightning-AI/torchmetrics

ИНСТРУКЦИИ ПО ПОТЕРЯМ : https://neptune.ai/blog/pytorch-loss-functions

Подбор гиперпараметров Optuna (Pytorch) : https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py

Optuna общий гайд : https://optuna.readthedocs.io/en/stable/tutorial/index.html